## Neighborhoods in Toronto

**Import All Libraries** 

In [6]:
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


**Fetch Data from Wikipedia site**

In [7]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(wiki_url)
soup = BeautifulSoup(page.text)
table_data = soup.find_all('table')[0]
row_data = table_data.find_all('tr')

**Create Neighborhood Data frame from Page data**

In [8]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for row in row_data:
    cols = row.find_all('td')
    if (len(cols)==3 and cols[1].string != 'Not assigned'):
        postalCode=cols[0].string
        borough=cols[1].string
        for name in cols[2].stripped_strings:
            neighborhood_name=name
        if(neighborhood_name=='Not assigned'):
            neighborhood_name=borough
            
        neighborhoods = neighborhoods.append({'PostalCode': postalCode,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood_name}, ignore_index=True)
neighborhoods = neighborhoods.groupby(['PostalCode','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)]).reset_index()

In [9]:
neighborhoods.shape

(103, 3)

In [10]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Read Geospatial Data**

In [11]:
postal_df = pd.read_csv('http://cocl.us/Geospatial_data')

**Merge Geospatial Data and Neighborhoods data**

In [12]:
neighborhoods=neighborhoods.merge(postal_df,left_on='PostalCode', right_on='Postal Code')
neighborhoods=neighborhoods.drop(['Postal Code'], axis=1)
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
